Process the unlabelled data.
</br>
We have 2 ideas. 
</br>
We will use multiple well-trained models and make the prediction of unlabelled data.

*   We average the prediction and get the label of the unlabelled data
*   We vote for mode after using the arguemax method and transform to the 0 and 1 vectors. Then we get the label of the unlabelled data.



In [ ]:
! gdown https://drive.google.com/uc?id=1AbxjCBByF-xGV3c7FJ1ztZC1HO0O0uJ0

Downloading...
From: https://drive.google.com/uc?id=1AbxjCBByF-xGV3c7FJ1ztZC1HO0O0uJ0
To: /content/kaggle.json
100% 66.0/66.0 [00:00<00:00, 150kB/s]


In [ ]:
! pip install kaggle
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
! kaggle competitions download -c comp-551-fall-2021
! unzip images_l.pkl.zip
! unzip images_test.pkl.zip
! unzip images_ul.pkl.zip
! unzip labels_l.pkl.zip

 90% 89.0M/99.0M [00:00<00:00, 89.1MB/s]
100% 99.0M/99.0M [00:01<00:00, 102MB/s] 
 96% 189M/197M [00:01<00:00, 168MB/s]
100% 197M/197M [00:01<00:00, 148MB/s]
 97% 193M/198M [00:01<00:00, 131MB/s]
100% 198M/198M [00:01<00:00, 118MB/s]
  0% 0.00/141k [00:00<?, ?B/s]
100% 141k/141k [00:00<00:00, 133MB/s]
Archive:  images_l.pkl.zip
  inflating: images_l.pkl            
Archive:  images_test.pkl.zip
  inflating: images_test.pkl         
Archive:  images_ul.pkl.zip
  inflating: images_ul.pkl           
Archive:  labels_l.pkl.zip
  inflating: labels_l.pkl            


In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU, ZeroPadding2D,Convolution2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import History
from tensorflow.keras.optimizers import Nadam, Adam, SGD
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet169
from keras.applications.resnet import ResNet152, ResNet101, ResNet50
import matplotlib.pyplot as plt
from keras.utils import np_utils
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras import layers, regularizers, Input
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from google.colab import drive
import h5py
import random
import torch

In [ ]:
# Image data for training (30,000 sample, each sample is a 56x56 image)
# train_labels = pd.read_csv('train_max_y.csv')
# train_images = pd.read_pickle('train_max_x')
# test_images = pd.read_pickle('test_max_x')
with open("images_l.pkl", 'rb') as f:
    # load into as a numpy array
    train_labelled_data = pickle.load(f)
# Test images. The prediction corresponding to these images should be uploaded. (15,000 samples)
with open("images_test.pkl", 'rb') as f:
    test_data = pickle.load(f)
# Labels for training (30,000 rows, each row is a size 36 binary vector, which is the label to the corresponding image)
with open("labels_l.pkl", 'rb') as f:
    train_labelled_y = pickle.load(f)
# Additional images that can be used for training the classifier.
# Labels for these images are not provided. (30,000 samples, where each sample is a 56x56 image)
with open("images_ul.pkl", 'rb') as f:
    train_unlabelled_x = pickle.load(f)

# processing

load processed train data for digit prediction

In [ ]:
# load process data
! gdown https://drive.google.com/uc?id=1wyUpGYONbWRjdaLgeiEH2ryqm_kMOMF_
! gdown https://drive.google.com/uc?id=1tFt0ddczfNPmq8IUNnZ8ifGUbgzP9DxU
! gdown https://drive.google.com/uc?id=1bOq70mP5ghxfiFtbK3rTbPDcefkxgyQx

Downloading...
From: https://drive.google.com/uc?id=1wyUpGYONbWRjdaLgeiEH2ryqm_kMOMF_
To: /content/processed_train_set.csv
100% 386M/386M [00:02<00:00, 179MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tFt0ddczfNPmq8IUNnZ8ifGUbgzP9DxU
To: /content/processed_test_set.csv
100% 193M/193M [00:01<00:00, 176MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bOq70mP5ghxfiFtbK3rTbPDcefkxgyQx
To: /content/processed_ul_train_set.csv
100% 386M/386M [00:02<00:00, 174MB/s]


Load best performance models

In [ ]:
# load models
! gdown https://drive.google.com/uc?id=1gpLzFm3QW1q__KMAg4QWe2pqVveAmd_y
! gdown https://drive.google.com/uc?id=1DLNI6xb1SNEFMcEcNgJb_q0WYR5OlKi2
! gdown https://drive.google.com/uc?id=1VFcPDEt_Ges0eJ7FB9iKoWV4goSMGqk2
! gdown https://drive.google.com/uc?id=1p_etGMx0I57VG6SGR_XBp3y153gNLHNR
! gdown https://drive.google.com/uc?id=1nO0-eQyOBKSe-VqK3jwQ6iCFy-tkf6xK

! gdown https://drive.google.com/uc?id=1YKExD85CaznZqjKOYPc3of08iCdMrUtz
! gdown https://drive.google.com/uc?id=186i8-O8q07Q2CrEz7O2GXaj9Yz9NKXwC
! gdown https://drive.google.com/uc?id=1-IIib1GFCDFYvtvCoME98P8VszmC9yQT
! gdown https://drive.google.com/uc?id=1ZdqmrMxnCz6NbJkYd5DfmGdQ_34t7A95
! gdown https://drive.google.com/uc?id=1cjlv8lHW1PtPdrS6S9-Myb0zaCrLzl8v


Downloading...
From: https://drive.google.com/uc?id=1gpLzFm3QW1q__KMAg4QWe2pqVveAmd_y
To: /content/VGG_valid_95.8.pkl
100% 61.4M/61.4M [00:00<00:00, 195MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DLNI6xb1SNEFMcEcNgJb_q0WYR5OlKi2
To: /content/VGG_valid_95.5.pkl
100% 61.4M/61.4M [00:00<00:00, 169MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VFcPDEt_Ges0eJ7FB9iKoWV4goSMGqk2
To: /content/VGG_valid_94.8.pkl
100% 61.4M/61.4M [00:00<00:00, 167MB/s]
Downloading...
From: https://drive.google.com/uc?id=1p_etGMx0I57VG6SGR_XBp3y153gNLHNR
To: /content/VGG_valid_95.4.pkl
100% 61.4M/61.4M [00:00<00:00, 160MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nO0-eQyOBKSe-VqK3jwQ6iCFy-tkf6xK
To: /content/model1.pkl
100% 183M/183M [00:01<00:00, 151MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YKExD85CaznZqjKOYPc3of08iCdMrUtz
To: /content/wmy_valid_97.9_1.pkl
100% 76.0M/76.0M [00:00<00:00, 92.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=186i8-O

In [ ]:
model1_1 = pickle.load(open("VGG_valid_95.8.pkl", 'rb'))
model1_2 = pickle.load(open("VGG_valid_94.8.pkl", 'rb'))
model1_3 = pickle.load(open("VGG_valid_95.4.pkl", 'rb'))
model1_4 = pickle.load(open("VGG_valid_95.5.pkl", 'rb'))
model1_5 = pickle.load(open("model1.pkl", 'rb'))

model2_1 = pickle.load(open("model2_valid_97.9.pkl", 'rb'))
model2_2 = pickle.load(open("model2_valid_97.9_1.pkl", 'rb'))
model2_3 = pickle.load(open("model2_valid_97.3.pkl", 'rb'))
model2_4 = pickle.load(open("model2_valid_97.5.pkl", 'rb'))
model2_5 = pickle.load(open("model2.pkl", 'rb'))

# digit

Load processed unlabelled data

In [ ]:
# load unlabelled data for model testing
x_test=np.array(pd.DataFrame(pd.read_csv('processed_ul_train_set.csv')))

x_test=x_test.reshape((30000,56,56))
# np.random.shuffle(x_test)
x_test=x_test[12000:15000]

# with open("images_test.pkl",'rb') as f:
#   x_test = pickle.load(f)
for i in range(x_test.shape[0]):
  x_test[i] = (x_test[i]>200).astype('int32')*255
x_test = torch.Tensor(x_test)
x_test = torch.unsqueeze(x_test,dim=3)/255
x_test = x_test.repeat(1,1,1,3)
x_test = x_test.numpy()
x_test = x_test.astype('float32')

In [ ]:
result_1 = model1_1.predict(x_test,verbose=1)
result_2 = model1_2.predict(x_test,verbose=1)
result_3 = model1_3.predict(x_test,verbose=1)
result_4 = model1_4.predict(x_test,verbose=1)
result_5 = model1_5.predict(x_test,verbose=1)

94/94 [==============================] - 1s 9ms/step


In [ ]:
# averaging
result = np.zeros((3000,10))
for i in range (0,result_1.shape[0]):
  for j in range (0,result_1.shape[1]):
    result[i][j] = (result_1[i][j] + result_2[i][j] + result_3[i][j] + result_4[i][j] + result_5[i][j])/5

result = np.argmax(result, axis = 1)
print(result[0:10])

[2 0 0 1 4 5 3 4 1 2]


In [ ]:
# mode
result_1 = np.argmax(result_1, axis = 1)
result_2 = np.argmax(result_2, axis = 1)
result_3 = np.argmax(result_3, axis = 1)
result_4 = np.argmax(result_4, axis = 1)
result_5 = np.argmax(result_5, axis = 1)

print(result_1[0:10])
print(result_2[0:10])
print(result_3[0:10])
print(result_4[0:10])
print(result_5[0:10])

[5 3 6 4 9 9 0 9 2 5]
[5 3 6 4 9 6 0 9 2 3]
[5 3 6 4 9 6 0 9 2 9]
[5 3 6 4 9 9 0 9 2 3]
[5 3 6 4 9 9 0 9 2 9]


In [ ]:
from scipy import stats
result = np.zeros(15000)
for i in range (result_1.shape[0]):
  array = np.array([result_1[i],result_2[i],result_3[i],result_4[i],result_5[i]])
  mode = stats.mode(array)
  result[i]=mode[0][0]
result = result.astype(int)
print(result[0:10])

[0 4 9 0 6 9 3 2 1 6]


# letter

Since model2 for letter prediction is not noise sensitive, so we do not need the processed unlabelled train set. 

In [ ]:
with open("images_ul.pkl", 'rb') as f:
    train_unlabelled_x = pickle.load(f)
x_test = x_test_data.reshape(-1,56,56,1)
# np.random.shuffle(x_test)
# x_test /= 255
x_test = x_test[12000:15000]

In [ ]:
result2 = model2.predict(x_test, verbose=1)
result2 = np.argmax(result2, axis = 1)
print(result2[0:10])

In [ ]:
result2_1 = model2_1.predict(x_test,verbose=1)
result2_2 = model2_2.predict(x_test,verbose=1)
result2_3 = model2_3.predict(x_test,verbose=1)
result2_4 = model2_4.predict(x_test,verbose=1)
result2_5 = model2_5.predict(x_test,verbose=1)

94/94 [==============================] - 1s 6ms/step


In [ ]:
# averaging
result2 = np.zeros((3000,26))
for i in range (0,result2_1.shape[0]):
  for j in range (0,result2_1.shape[1]):
    result2[i][j] = (result2_1[i][j] + result2_2[i][j] + result2_3[i][j] + result2_4[i][j]+ result2_5[i][j])/5

result2 = np.argmax(result2, axis = 1)
print(result2[0:10])

[ 6 18  9 13 20  0 11 17 22  5]


In [ ]:
# mode
result2_1 = np.argmax(result2_1, axis = 1)
result2_2 = np.argmax(result2_2, axis = 1)
result2_3 = np.argmax(result2_3, axis = 1)
result2_4 = np.argmax(result2_4, axis = 1)
result2_5 = np.argmax(result2_5, axis = 1)

print(result2_1[0:10])
print(result2_2[0:10])
print(result2_3[0:10])
print(result2_4[0:10])
print(result2_5[0:10])

from scipy import stats
result2 = np.zeros(15000)
for i in range (result2_1.shape[0]):
  array = np.array([result2_1[i],result2_2[i],result2_3[i],result2_4[i],result2_5[i]])
  mode = stats.mode(array)
  result2[i]=mode[0][0]
result2 = result2.astype(int)
print(result2[0:10])


[19 11  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]
[19  7  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]


# output

We want to output a csv that contains number of records of size of unlabelled set, and each record is a 36 dimension vector constitute by 0s and 1s.

In [ ]:
def transform_digit(num):
    temp_list = np.zeros(10)
    temp_list[num] = 1
    temp_list = temp_list.astype(int)
    return temp_list


# tranform 0-25 number to 26-bit string with the index at the num value to be 1
def transform_letter(num):
    temp_list = np.zeros(26)
    temp_list[num] = 1
    temp_list = temp_list.astype(int)
    return temp_list

label_prediction = np.zeros((3000,36))
for i in range (0,3000):
    a=transform_digit(result[i])
    b=transform_letter(result2[i])
    label_prediction[i] = np.concatenate((a, b), axis=0)
label_prediction= label_prediction.astype(int)
print(label_prediction[0:10])
print(label_prediction.shape)
pd.DataFrame(label_prediction).to_csv('ul_train_label.csv',index=False)

[[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
(3000, 36)
